<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

In [11]:
!pip install gdown

In [12]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [13]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

resnet50_50-epochs_stl10 1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu


In [ ]:
# download and extract model files
os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
os.system('unzip {}'.format(folder_name))#googleドライブからダウンロードしたものを回答-関係なし
!ls

checkpoint_0040.pth.tar
config.yml
events.out.tfevents.1610927742.4cb2c837708d.2694093.0
resnet50_50-epochs_stl10.zip
sample_data
training.log


In [2]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
print(parent_dir)

Using device: cuda
/home/shirota/SimCLR


In [ ]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  data_dir = parent_dir+"/datasets"
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [4]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  data_dir = parent_dir+"/datasets"
  train_dataset = datasets.STL10(data_dir, split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10(data_dir, split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [10]:
config_yml = "./config2.yml"
with open(os.path.join(config_yml)) as file:
  config = yaml.load(file, Loader=yaml.SafeLoader)

print(config["arch"])

resnet18


In [6]:
if config.arch == 'resnet18':
  model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
elif config.arch == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

AttributeError: 'dict' object has no attribute 'arch'

In [11]:
if config["arch"] == 'resnet18':
  model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
elif config["arch"] == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [12]:
model_filename=parent_dir+"/"+config["model_dir"]#SimCLR/runs/...
checkpoint = torch.load(model_filename, map_location=device)
#checkpoint = torch.load('checkpoint_0040.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):#MLP層以外を学習済みパラメータで初期化するため、その部分のパラメータの名前からbackboneを除外

  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [13]:
log = model.load_state_dict(state_dict, strict=False)#strict=Falseなのでパラメータの名前が一致しない部分は無視
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [ ]:
if config.dataset_name == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config.dataset_name == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)#
print("Dataset:", config.dataset_name)

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified
Dataset: stl10


In [15]:
if config["dataset_name"] == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config["dataset_name"] == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)#
print("Dataset:", config["dataset_name"])

Dataset: stl10


In [16]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [18]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [19]:
epochs = 100
best_top1_accuracy = 0
best_top5_accuracy = 0
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")
  if epoch == 0:
    best_top1_accuracy = top1_accuracy
    best_top5_accuracy = top5_accuracy
  else:
    if top1_accuracy > best_top1_accuracy:
      best_top1_accuracy = top1_accuracy
      best_top5_accuracy = top5_accuracy

print(f"best top1 accuracy = {best_top1_accuracy.item()}, top5 accuracy at the time = {best_top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 19.69554328918457	Top1 Test accuracy: 34.2138671875	Top5 test acc: 83.82568359375
Epoch 1	Top1 Train accuracy 43.26976013183594	Top1 Test accuracy: 49.63623046875	Top5 test acc: 93.056640625
Epoch 2	Top1 Train accuracy 53.80859375	Top1 Test accuracy: 55.9619140625	Top5 test acc: 95.52734375
Epoch 3	Top1 Train accuracy 58.863739013671875	Top1 Test accuracy: 59.2236328125	Top5 test acc: 96.396484375
Epoch 4	Top1 Train accuracy 61.187957763671875	Top1 Test accuracy: 60.830078125	Top5 test acc: 96.91162109375
Epoch 5	Top1 Train accuracy 62.741268157958984	Top1 Test accuracy: 62.03369140625	Top5 test acc: 97.20458984375
Epoch 6	Top1 Train accuracy 63.795955657958984	Top1 Test accuracy: 63.17138671875	Top5 test acc: 97.46337890625
Epoch 7	Top1 Train accuracy 64.53585052490234	Top1 Test accuracy: 64.05517578125	Top5 test acc: 97.63427734375
Epoch 8	Top1 Train accuracy 65.4710464477539	Top1 Test accuracy: 64.384765625	Top5 test acc: 97.63427734375
Epoch 9	Top1 Train

In [24]:
import re

In [28]:
keyword = "DEBUG"
log_file = parent_dir+"/runs/Jun09_17-40-46_rock9/training.log"
top1_accuracy = 0
count = 0
with open(log_file, "r", encoding="utf-8") as f:
    for line in f:
        if keyword in line:
            line = line.strip()
            match = re.search(r"Top1 accuracy:\s*([0-9.]+)", line)

            if match:
                accuracy = float(match.group(1))
                if count == 0:
                    top1_accuracy = accuracy
                else:
                    if accuracy > top1_accuracy:
                        top1_accuracy = accuracy
                
                #print(f"Top1 accuracy: {accuracy}")
            else:
                print("Top1 accuracy の値が見つかりませんでした")
        count += 1
            #print(line.strip())

print(f"best top1 accuracy = {top1_accuracy}")


best top1 accuracy = 64.375
